In [18]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('techcrunch_posts.csv')

In [20]:
cat_start_data = data[data['category'] == 'Startups']

### TODO:
- Create time series graph of "hype/PR"
- Extract company names and group them with their respective posts
- Create sentiment analysis on each post within the company group
- Graph the sentiment analysis against outcome (if exists) "funded/exits/failures"